Importation des library

In [ ]:
import pandas as pd
import numpy as np
import vectorbt as vbt
import matplotlib.pyplot as plt
import anywidget
import os
plt.style.use('dark_background')

Chargement des données

In [ ]:
# Télécharger les données (Utilise Python Binance)
# a PARTIR D'une date jusqu'a aujourd'hui (intervale 15 minutes)
symbol = ['ETHUSDT']
#day
time = "1m"
# Téléchargement des données
data = vbt.BinanceData.download(
    symbol, 
    start='2025-03-25', 
    interval=time,
    tz_convert='Europe/Paris',
)

Mise en forme des données

In [ ]:
# Agencement des données + index + mis à l'heure de paris
df_raw = data.get(['Close','Open','High','Low','Volume']).copy()
df_raw.tail()

In [ ]:
# Separation des données en train et test (IS/OOS) (70/30%)
split_idx = int(0.7*len(df_raw))
train_data = df_raw.iloc[:split_idx]
test_data = df_raw.iloc[split_idx:]

# Affichage de la séparation des données
plt.figure(figsize=(10, 1))
plt.barh(['Dataset'], [len(df_raw)], color='lightgray', alpha=0.3)
plt.barh(['Dataset'], [split_idx], color='blue', alpha=0.5, label='Train')
plt.barh(['Dataset'], [len(df_raw)-split_idx], left=split_idx, color='orange', alpha=0.5, label='Test')
plt.title('Train/Test Split Visualization')
plt.legend()
plt.show()

In [ ]:
# Selection du train_set comme données d'entrainement
df = test_data.copy()

In [ ]:
import vectorbt as vbt
import plotly.graph_objects as go

close = df['Close']

# ================================================
# 1) EMA
# ================================================
ema9  = vbt.MA.run(close, window=9, ewm=True).ma
ema21 = vbt.MA.run(close, window=21, ewm=True).ma
ema50 = vbt.MA.run(close, window=50, ewm=True).ma

# ================================================
# 2) ATR
# ================================================
atr = vbt.ATR.run(
    high=df['High'],
    low=df['Low'],
    close=close,
    window=14
).atr

tp_atr = 1 * atr / close   # TP relatif

# ================================================
# 3) Signaux
# ================================================
entries = ema9.vbt.crossed_above(ema50)
exits   = ema9.vbt.crossed_below(ema50)

# ================================================
# 4) Backtest
# ================================================
pf_atr = vbt.Portfolio.from_signals(
    close,
    entries,
    exits,
    tp_stop=tp_atr,
    init_cash=10_000,
    fees=0.0004,
    slippage=0.0002,
    freq=time
)

print("===== ATR TP =====")
print(pf_atr.stats())

# ================================================
# 5) Graphique
# ================================================
trades = pf_atr.trades.records_readable



fig = go.Figure()

# Bougies
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=close,
    name="Prix",
    increasing_line_color='rgba(0,200,0,0.6)',
    decreasing_line_color='rgba(200,0,0,0.6)',
    increasing_fillcolor='rgba(0,200,0,0.3)',
    decreasing_fillcolor='rgba(200,0,0,0.3)'
))

# EMA
fig.add_trace(go.Scatter(x=df.index, y=ema9,  name="EMA 9",  line=dict(color='green')))
fig.add_trace(go.Scatter(x=df.index, y=ema21, name="EMA 21", line=dict(color='yellow')))
fig.add_trace(go.Scatter(x=df.index, y=ema50, name="EMA 50", line=dict(color='orange')))

# Bande TP ATR (projection)
fig.add_trace(go.Scatter(
    x=df.index,
    y=close + 1.2 * atr,
    name="TP ATR (1.2x)",
    line=dict(color='cyan', dash='dot')
))

# Entrées / sorties
fig.add_trace(go.Scatter(
    x=trades['Entry Timestamp'],
    y=trades['Avg Entry Price'],
    mode='markers',
    marker=dict(color='lime', size=9),
    name='Buy'
))

fig.add_trace(go.Scatter(
    x=trades['Exit Timestamp'],
    y=trades['Avg Exit Price'],
    mode='markers',
    marker=dict(color='red', size=9),
    name='Sell'
))

fig.update_layout(
    title="EMA Cross + TP ATR",
    template="plotly_dark",
    height=800,
    xaxis_rangeslider_visible=False
)

fig.show()
pf_atr.plot().show()
